### pip install & import

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
#import pandas_profiling
import datetime

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

# from xgboost import XGBRFRegressor
from sklearn.model_selection import cross_validate

### 최종데이터 불러오기

In [169]:
# movie_final_회귀 = pd.read_csv('../3_merge_top300/movie_final_reg.csv',encoding='utf-8')
movie_final_분류 = pd.read_csv('../3_merge_top300/movie_final_cls.csv',encoding='utf-8')

## 원핫인코딩 

In [170]:
# 원핫인코딩

# movie_labels = pd.get_dummies(movie_final,columns = ['국적','장르','등급','계절'])

In [171]:
# 독립 변수와 종속 변수 나눔

# 독립 = movie_DL[['감독_흥행', '배급사_흥행', '국적_기타', '국적_독일', '국적_미국',
#        '국적_영국', '국적_일본', '국적_중국', '국적_프랑스', '국적_한국', '장르_SF', '장르_가족', '장르_공연',
#        '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스', '장르_뮤지컬',
#        '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)', '장르_성인물(에로)', '장르_스릴러',
#        '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지',
#        '등급_12세관람가', '등급_15세관람가', '등급_전체관람가', '등급_청소년관람불가', '계절_가을', '계절_겨울',
#        '계절_봄', '계절_여름']] 
# 종속 = movie_DL[['전국 관객수']] 

# 스케일링

In [172]:
from sklearn.preprocessing import RobustScaler

# 객체생성
robust_scaler = RobustScaler()

# *** 만약 특정 열의 스케일링을 하고 싶은 경우 ***
A_n = robust_scaler.fit_transform(movie_final_분류['전국 스크린수'].values.reshape(-1,1))
movie_final_분류.insert(0, '전국 스크린수_scaled', A_n)
# movie_final_분류.drop(['전국 스크린수'], axis=1, inplace=False)

A_n1 = robust_scaler.fit_transform(movie_final_분류['총 관객수'].values.reshape(-1,1))
movie_final_분류.insert(0, '총 관객수_scaled', A_n1)
# movie_final_분류.drop(['총 관객수'], axis=1, inplace=False)

In [173]:
movie_final_분류.columns

Index(['총 관객수_scaled', '전국 스크린수_scaled', 'Unnamed: 0', '순번', '영화명', '감독',
       '배급사', '개봉일', '영화형태', '국적', '전국 스크린수', '전국 매출액', '전국 관객수', '서울 매출액',
       '서울 관객수', '장르', '등급', '영화구분', '연도', '월', '계절', '총 관객수', '코로나', '출연',
       '평점', '감독_흥행', '배급사_흥행', '주연배우_흥행'],
      dtype='object')

## 라벨링

In [174]:
from sklearn.preprocessing import LabelEncoder

le_form = LabelEncoder()
le_nation = LabelEncoder()
le_genre = LabelEncoder()
le_rating = LabelEncoder()
le_class = LabelEncoder()
le_season = LabelEncoder()

movie_labels = movie_final_분류.copy()
movie_labels['영화형태'] = le_form.fit_transform(movie_final_분류['영화형태'])
movie_labels['국적'] = le_nation.fit_transform(movie_final_분류['국적'])
movie_labels['장르'] = le_genre.fit_transform(movie_final_분류['장르'])
movie_labels['등급'] = le_rating.fit_transform(movie_final_분류['등급'])
movie_labels['영화구분'] = le_class.fit_transform(movie_final_분류['영화구분'])
movie_labels['계절'] = le_season.fit_transform(movie_final_분류['계절'])
movie_labels.drop(['Unnamed: 0'],axis=1,inplace=True)
movie_labels.head(3)


,총 관객수_scaled,전국 스크린수_scaled,순번,영화명,감독,배급사,개봉일,영화형태,국적,전국 스크린수,...,연도,월,계절,총 관객수,코로나,출연,평점,감독_흥행,배급사_흥행,주연배우_흥행
0,0.027876,1.397641,1,명량,김한민,씨제이이엔엠,2014-07-30,0,7,1587,...,2014,7,3,215068958,0,"['최민식', '류승룡', '조진웅']",8.44,9,10,20
1,0.216263,2.056445,2,극한직업,이병헌,씨제이이엔엠,2019-01-23,0,7,1978,...,2019,1,1,226678777,0,"['류승룡', '이하늬', '진선규', '이동휘', '공명']",8.51,4,10,16
2,0.105881,1.945240,3,신과함께-죄와 벌,김용화,롯데엔터테인먼트,2017-12-20,0,7,1912,...,2017,12,1,219876227,0,"['하정우', '차태현', '주지훈', '김향기', '김동욱', '마동석']",7.83,9,9,40


In [92]:
# movie_labels[movie_labels['연도']!=2023]

### 분류 클래스 컬럼 추가하기

In [ ]:
display(
    len(train_target[train_target['전국 관객수']<1000000]),
    len(train_target[(train_target['전국 관객수']>1000000) & (train_target['전국 관객수']<1400000)]),
    len(train_target[(train_target['전국 관객수']>1400000) & (train_target['전국 관객수']<2000000)]),
    len(train_target[(train_target['전국 관객수']>2000000) & (train_target['전국 관객수']<3000000)]),
    len(train_target[(train_target['전국 관객수']>3000000) & (train_target['전국 관객수']<5000000)]),
    len(train_target[train_target['전국 관객수']>5000000])
)

59

88

88

92

68

57

In [175]:
import numpy as np

movie_labels['분류클래스'] = np.where((movie_labels['전국 관객수'].values<1000000), 0, 
                                 np.where((movie_labels['전국 관객수'].values>1000000) & (movie_labels['전국 관객수'].values<1400000), 1,
                                          np.where((movie_labels['전국 관객수'].values>1400000) & (movie_labels['전국 관객수'].values<2000000), 2,
                                                   np.where((movie_labels['전국 관객수'].values>2000000) & (movie_labels['전국 관객수'].values<3000000), 3,
                                                            np.where((movie_labels['전국 관객수'].values>3000000) & (movie_labels['전국 관객수'].values<5000000), 4, 5))))
)
movie_labels['분류클래스'].unique()
movie_labels[['전국 관객수','분류클래스']]
movie_labels['분류클래스'].value_counts()

3    137
2    131
1    130
4    109
5    101
0    100
Name: 분류클래스, dtype: int64

### 데이터분배
1) movie_labels_etc : train과 val data   
2) movie_labels_2023 : test data

In [93]:
# movie_labels_2023 = movie_labels[movie_labels['연도']==2023]
# movie_labels_etc = movie_labels[movie_labels['연도']!=2023]
# display(movie_labels_etc.head(),
# movie_labels_2023.head())

### 변수 정리   
1. 사용하지 않는 컬럼 제거
2. 독립변수와 종속변수 설정

In [94]:
# # 사용하지 않는 컬럼 제거

# not_used = ['영화명', '감독', '배급사', '영화형태', '개봉일', '영화구분', '월']

# movie_DL = movie_labels_etc.drop(not_used, axis=1)      # 2003~2022 데이터 main
# movie_DL2 = movie_labels_2023.drop(not_used, axis=1)    # 2023년 test


In [176]:
# 사용하지 않는 컬럼 제거

not_used = ['영화명', '감독', '배급사', '영화형태', '개봉일', '영화구분', '월']

movie_DL = movie_labels.drop(not_used, axis=1)
movie_DL.head()

,총 관객수_scaled,전국 스크린수_scaled,순번,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,...,연도,계절,총 관객수,코로나,출연,평점,감독_흥행,배급사_흥행,주연배우_흥행,분류클래스
0,0.027876,1.397641,1,7,1587,135748398910,17613682,"33,121,225,810","4,163,666",9,...,2014,3,215068958,0,"['최민식', '류승룡', '조진웅']",8.44,9,10,20,5
1,0.216263,2.056445,2,7,1978,139647979516,16264944,"31,858,660,536","3,638,287",16,...,2019,1,226678777,0,"['류승룡', '이하늬', '진선규', '이동휘', '공명']",8.51,4,10,16,5
2,0.105881,1.945240,3,7,1912,115698654137,14410754,"27,530,825,087","3,346,172",17,...,2017,1,219876227,0,"['하정우', '차태현', '주지훈', '김향기', '김동욱', '마동석']",7.83,9,9,40,5
3,0.027876,0.351306,4,7,966,110828014630,14245998,"25,842,519,330","3,233,946",4,...,2014,1,215068958,0,"['황정민', '김윤진', '오달수', '정진영']",9.02,10,10,17,5
4,0.216263,3.500421,5,2,2835,122182694160,13934592,"33,577,136,860","3,597,963",13,...,2019,2,226678777,0,"['로버트 다우니 주니어', '크리스 에반스', '크리스 헴스워스', '마크 러팔로...",9.38,10,10,106,5


In [96]:
# 국적 => 국내 vs 해외
# 관객비율.. 2023년 데이터 총 관객수 몰라....
# 비율 말고 총 관객수를 쓰자
# 총관객수 defqult를 2022년으로 해서 2023년 총관객수 채우면 어때?

In [97]:
# # 독립 변수와 종속 변수 나눔

# 종속 = movie_DL[['전국 관객수']].copy()
# 종속['전국 관객수'] = 종속['전국 관객수'] / 1000 

# 종속_2023 = movie_DL2[['전국 관객수']].copy()
# 종속_2023['전국 관객수'] = 종속_2023['전국 관객수'] / 1000 


# 독립 = movie_DL[['장르', '등급', '감독_흥행','총 관객수_scaled',
#                '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

# 독립_2023 = movie_DL2[['장르', '등급', '감독_흥행','총 관객수_scaled',
#                '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

# display(독립.head(), 독립_2023.head())

In [207]:
독립 = movie_DL[['장르', '등급', '감독_흥행','총 관객수_scaled',
               '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','코로나']].copy()

종속 = movie_DL[['분류클래스']].copy()

display(독립.info(), 종속['분류클래스'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 708 entries, 0 to 707
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   장르              708 non-null    int32  
 1   등급              708 non-null    int32  
 2   감독_흥행           708 non-null    int64  
 3   총 관객수_scaled    708 non-null    float64
 4   배급사_흥행          708 non-null    int64  
 5   주연배우_흥행         708 non-null    int64  
 6   계절              708 non-null    int32  
 7   전국 스크린수_scaled  708 non-null    float64
 8   코로나             708 non-null    int64  
dtypes: float64(2), int32(3), int64(4)
memory usage: 41.6 KB


None

array([5, 4, 3, 2, 1, 0])

## train, test 데이터 나누기

In [208]:
# train / test 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(독립, 종속, test_size=0.20, random_state=42)

In [209]:
# train / val 나누기
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.20, random_state=42)
print(len(train_input), len(val_input), len(test_input))

452 114 142


In [210]:
# # train, validation 데이터 나눔

# from sklearn.model_selection import train_test_split
# train_input, test_input, train_target, test_target = train_test_split(독립,종속,test_size=0.25,random_state=42)

## target 클래스 분류하기

In [211]:
# display(
#     len(train_target[train_target['전국 관객수']<1000000]),
#     len(train_target[(train_target['전국 관객수']>1000000) & (train_target['전국 관객수']<1400000)]),
#     len(train_target[(train_target['전국 관객수']>1400000) & (train_target['전국 관객수']<2000000)]),
#     len(train_target[(train_target['전국 관객수']>2000000) & (train_target['전국 관객수']<3000000)]),
#     len(train_target[(train_target['전국 관객수']>3000000) & (train_target['전국 관객수']<5000000)]),
#     len(train_target[train_target['전국 관객수']>5000000])
# )

In [212]:
# # train 분류 클래스 만들기

# import numpy as np

# train_target['분류클래스'] = np.where((train_target['전국 관객수'].values<1000000), 1, 
#                                  np.where((train_target['전국 관객수'].values>1000000) & (train_target['전국 관객수'].values<1400000), 2,
#                                           np.where((train_target['전국 관객수'].values>1400000) & (train_target['전국 관객수'].values<2000000), 3,
#                                                    np.where((train_target['전국 관객수'].values>2000000) & (train_target['전국 관객수'].values<3000000), 4,
#                                                             np.where((train_target['전국 관객수'].values>3000000) & (train_target['전국 관객수'].values<5000000), 5, 6))))
# )
# train_target['분류클래스'].unique()
# train_target[['전국 관객수','분류클래스']]
# train_target['분류클래스'].value_counts()

In [213]:
# # val 분류 클래스 만들기
# val_target['분류클래스'] = np.where((val_target['전국 관객수'].values<1000000), 1, 
#                                  np.where((val_target['전국 관객수'].values>1000000) & (val_target['전국 관객수'].values<1400000), 2,
#                                           np.where((val_target['전국 관객수'].values>1400000) & (val_target['전국 관객수'].values<2000000), 3,
#                                                    np.where((val_target['전국 관객수'].values>2000000) & (val_target['전국 관객수'].values<3000000), 4,
#                                                             np.where((val_target['전국 관객수'].values>3000000) & (val_target['전국 관객수'].values<5000000), 5, 6))))
# )
# val_target['분류클래스'].unique()
# # val_target[['전국 관객수','분류클래스']]

In [214]:
# # test 분류 클래스 만들기

# test_target['분류클래스'] = np.where((test_target['전국 관객수'].values<1000000), 1, 
#                                  np.where((test_target['전국 관객수'].values>1000000) & (test_target['전국 관객수'].values<1400000), 2,
#                                           np.where((test_target['전국 관객수'].values>1400000) & (test_target['전국 관객수'].values<2000000), 3,
#                                                    np.where((test_target['전국 관객수'].values>2000000) & (test_target['전국 관객수'].values<3000000), 4,
#                                                             np.where((test_target['전국 관객수'].values>3000000) & (test_target['전국 관객수'].values<5000000), 5, 6))))
# )
# test_target['분류클래스'].unique()
# # test_target[['전국 관객수','분류클래스']]

In [215]:
# # 값 스케일링

# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# ss.fit(train_input)
# train_scaled = ss.transform(train_input)
# test_scaled = ss.transform(test_input)

In [216]:
train_target.head()

,분류클래스
590,1
341,3
522,1
531,1
567,1


# 딥러닝 

In [217]:
# # 딥러닝 하기 전 종속변수를 위에서 만든 분류 클래스로 정의하기
# train_target = train_target.drop(['전국 관객수'], axis=1)
# val_target = val_target.drop(['전국 관객수'], axis=1)
# test_target = test_target.drop(['전국 관객수'], axis=1)

# train_target.head()


In [218]:
print(독립.shape, 종속.shape)

(708, 9) (708, 1)


In [186]:
train_input.columns
train_target

,분류클래스
590,1
341,3
522,1
531,1
567,1
...,...
396,2
257,3
374,2
596,1


In [203]:
# 딥러닝 층 여기가 중요

X = tf.keras.layers.Input(shape=[9])

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization()(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

# H = tf.keras.layers.Dense(64)(H)
# H = tf.keras.layers.BatchNormalization()(H)
# H = tf.keras.layers.Activation(tf.nn.swish)(H)

# H = tf.keras.layers.Dense(128)(H)
# H = tf.keras.layers.BatchNormalization()(H)
# H = tf.keras.layers.Activation(tf.nn.swish)(H)
# # dropout = tf.keras.layers.Dropout(rate=0.3)(H)

Y = tf.keras.layers.Dense(6, activation='softmax')(H)
model = tf.keras.models.Model(X,Y)
#sgd = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.8,beta_2=0.85)
#sgd = tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.7)
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [204]:
# 훈련
history = model.fit(train_input,train_target,batch_size=128,epochs=100,validation_data=(val_input,val_target))

Epoch 1/100
4/4 [==============================] - 1s 43ms/step - loss: 1.8156 - accuracy: 0.1704 - val_loss: 194.5861 - val_accuracy: 0.1579
Epoch 2/100
4/4 [==============================] - 0s 7ms/step - loss: 1.7359 - accuracy: 0.2434 - val_loss: 193.4939 - val_accuracy: 0.1667
Epoch 3/100
4/4 [==============================] - 0s 19ms/step - loss: 1.7177 - accuracy: 0.2588 - val_loss: 180.6337 - val_accuracy: 0.1667
Epoch 4/100
4/4 [==============================] - 0s 10ms/step - loss: 1.6989 - accuracy: 0.2478 - val_loss: 174.8022 - val_accuracy: 0.1667
Epoch 5/100
4/4 [==============================] - 0s 8ms/step - loss: 1.6891 - accuracy: 0.2810 - val_loss: 161.4308 - val_accuracy: 0.1667
Epoch 6/100
4/4 [==============================] - 0s 9ms/step - loss: 1.6731 - accuracy: 0.2788 - val_loss: 150.0662 - val_accuracy: 0.1667
Epoch 7/100
4/4 [==============================] - 0s 12ms/step - loss: 1.6628 - accuracy: 0.2810 - val_loss: 137.2366 - val_accuracy: 0.1667
Epoch 8/1

In [205]:
# 확인용도
model.predict(test_input[:5])

1/1 [==============================] - 0s 81ms/step


array([[7.4423831e-26, 1.9491239e-03, 1.7569855e-25, 9.9805093e-01,
        5.5433343e-28, 0.0000000e+00],
       [1.0319246e-25, 3.5932749e-03, 3.1549227e-25, 9.9640673e-01,
        6.4006649e-28, 0.0000000e+00],
       [6.9597099e-27, 3.2775739e-04, 9.7789663e-26, 9.9967229e-01,
        2.8292662e-28, 0.0000000e+00],
       [5.4355116e-28, 2.9068413e-05, 3.8215494e-26, 9.9997091e-01,
        1.2607311e-28, 0.0000000e+00],
       [8.7456610e-27, 5.5289350e-04, 1.0739190e-25, 9.9944705e-01,
        4.7612816e-29, 0.0000000e+00]], dtype=float32)

In [206]:
# 여기랑 비슷한지 보기
test_target[:5]

,분류클래스
120,4
247,3
324,3
204,4
603,1


### test data 확인용도

In [191]:
# 확인용도

# 오차율 몇 %로...?
pred = model.predict(독립_2023).round(0)
# 여기랑 비슷한지 보기
종속_2023['pred'] = pred
종속_2023['오차율'] = (abs(종속_2023['전국 관객수'] - 종속_2023['pred'])) / 종속_2023['전국 관객수'] * 100
len(종속_2023.loc[종속_2023['오차율']<20])

NameError: name '독립_2023' is not defined

In [42]:
종속_2023

,전국 관객수,pred,오차율
398,1720.865,2441.0,41.847269
485,1376.493,4026.0,192.482417
690,829.268,1706.0,105.723602
